# K-nearest neighbor

In [1]:
# Data import
from preprocessing.preprocessing import download_brfss_dataset
#download_brfss_dataset("username", "token") -> insert kaggle username and api-token

In [1]:
#imports

from preprocessing.preprocessing import get_preprocessed_brfss_train_test_split_one_hot_encoded, get_preprocessed_brfss_train_test_split, get_preprocessed_brfss_dataset

import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV


In [8]:
train_dataset, target = get_preprocessed_brfss_dataset()
target = target.squeeze()

0    3.0
1    3.0
2    3.0
3    3.0
4    3.0
Name: DIABETE3, dtype: float64

In [2]:
data_train, data_test, target_train, target_test = get_preprocessed_brfss_train_test_split()
data_train = data_train.head(5000)
target_train = target_train.head(5000)

In [11]:
for n_neighbors in range(1,10):
    knn_estimator = KNeighborsClassifier(n_neighbors)
    knn_estimator.fit(data_train, target_train)
    diabetes_test_prediction = knn_estimator.predict(data_test)
    print("k= {} acc: {}".format(n_neighbors, accuracy_score(target_test, diabetes_test_prediction)))

k= 1 acc: 0.7655227092535961
k= 2 acc: 0.6503114735530637
k= 3 acc: 0.807894438781289
k= 4 acc: 0.7589874277947672
k= 5 acc: 0.8235587269226413
k= 6 acc: 0.7967833276701778
k= 7 acc: 0.8309548080190282
k= 8 acc: 0.8142258466417488
k= 9 acc: 0.8346471854117115


In [4]:
knn_estimator = KNeighborsClassifier(7)
knn_estimator.get_params()

params = {
    'n_neighbors': range (7,15),
    'metric': ('minkowski', 'cosine', 'sqeuclidean')
}

grid_search_estimator = GridSearchCV(knn_estimator, params, scoring='accuracy', cv=None, return_train_score=False)
grid_search_estimator.fit(data_train,target_train.values.ravel())

results = pd.DataFrame(grid_search_estimator.cv_results_)
display(results)


print("best score is {} with params {}".format(grid_search_estimator.best_score_, grid_search_estimator.best_params_))

/Users/I516258/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:684: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_metric,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.005359,0.000347,0.068220,0.011216,minkowski,7,"{'metric': 'minkowski', 'n_neighbors': 7}",0.827,0.829,0.828,0.836,0.821,0.8282,0.004792,23
1,0.006078,0.000367,0.130226,0.073744,minkowski,8,"{'metric': 'minkowski', 'n_neighbors': 8}",0.825,0.830,0.825,0.835,0.829,0.8288,0.003709,21
2,0.007070,0.002043,0.084790,0.012640,minkowski,9,"{'metric': 'minkowski', 'n_neighbors': 9}",0.830,0.838,0.834,0.841,0.830,0.8346,0.004363,15
3,0.006369,0.000824,0.074425,0.010111,minkowski,10,"{'metric': 'minkowski', 'n_neighbors': 10}",0.831,0.836,0.836,0.844,0.833,0.8360,0.004427,13
4,0.005695,0.000290,0.061175,0.001243,minkowski,11,"{'metric': 'minkowski', 'n_neighbors': 11}",0.836,0.841,0.839,0.843,0.836,0.8390,0.002757,3
5,0.005579,0.000397,0.060815,0.002598,minkowski,12,"{'metric': 'minkowski', 'n_neighbors': 12}",0.837,0.840,0.839,0.842,0.838,0.8392,0.001720,1
6,0.005443,0.000161,0.059589,0.000812,minkowski,13,"{'metric': 'minkowski', 'n_neighbors': 13}",0.837,0.840,0.839,0.842,0.837,0.8390,0.001897,3
7,0.005301,0.000106,0.060239,0.001776,minkowski,14,"{'metric': 'minkowski', 'n_neighbors': 14}",0.837,0.839,0.841,0.841,0.837,0.8390,0.001789,9
8,0.005122,0.000059,0.095571,0.004623,cosine,7,"{'metric': 'cosine', 'n_neighbors': 7}",0.830,0.827,0.822,0.839,0.835,0.8306,0.005953,19
9,0.005330,0.000296,0.095449,0.003272,cosine,8,"{'metric': 'cosine', 'n_neighbors': 8}",0.826,0.824,0.826,0.834,0.834,0.8288,0.004308,20


best score is 0.8392 with params {'metric': 'minkowski', 'n_neighbors': 12}
